# Creating Analytical ready delta tables

Create "analytics" schema 

In [0]:
%sql

CREATE DATABASE IF NOT EXISTS analytics;

In [0]:
# To read staging delta table
df = spark.read.table('staging.food_delivery_stage')


**1. Performance summary of each delivery person**

In [0]:
# 1. Performance summary of each delivery person
delivery_person_performance_summary = df.groupBy('delivery_person_id').agg(
    sum('time_taken_min').alias('total_delivery_time_min'),  # Total delivery time
    count('id').alias('total_deliveries'),                   # Total number of deliveries
    avg('time_taken_min').alias('avg_delivery_time_min'),        # Average delivery time
    max('time_taken_min').alias('max_delivery_time_min'),        # Max delivery time
    min('time_taken_min').alias('min_delivery_time_min'),        # Min delivery time
    max('multiple_deliveries').alias('max_single_trip_delivery'),       # Max multiple deliveries in a single trip
    min('multiple_deliveries').alias('min_single_trip_delivery'),         # Max multiple deliveries in a single trip
    avg('delivery_person_ratings').alias('avg_ratings'),
    count(when(col("multiple_deliveries") > 1, 1)).alias('extra_credit_deliveries')  # number of times extra deliveries made on a single trip
)

# Load the results to delta table
delivery_person_performance_summary.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("analytics.delivery_person_performance_summary")
               


**2. Delivery summary**

In [0]:
from pyspark.sql.functions import *

# 2. Delivery summary 
# Create the delivery performance summary, now grouped by city and year
delivery_performance_summary = df.groupBy('year', 'month', 'city').agg(
    avg('time_taken_min').alias('avg_delivery_time'),
    min('time_taken_min').alias('min_delivery_time'),
    max('time_taken_min').alias('max_delivery_time'),
    count('id').alias('total_deliveries')
)

# Clean the 'type_of_order' column: remove whitespaces and convert to lowercase
order_type_counts_cleaned = order_type_counts.withColumn(
    'type_of_order', lower(trim(col('type_of_order')))
)

# Pivot the data to count each type of order separately per city and year
order_type_counts = df.groupBy('year', 'month', 'city', 'type_of_order').agg(
    count('id').alias('order_type_count')
)

# Pivot to get separate columns for each order type (dynamically, based on the type_of_order)
pivot_order_types = order_type_counts_cleaned.groupBy('year', 'month', 'city').pivot('type_of_order').agg(
    first('order_type_count')  # We use 'first' as there should be only one count per city/year/order type
)

# Join the performance summary with the pivoted order type counts
final_summary = delivery_performance_summary.join(pivot_order_types, on=['year', 'month', 'city'], how='left')

# Load the results to delta table
final_summary.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("analytics.delivery_summary_by_month_city")


In [0]:
display(final_summary)

year,month,city,avg_delivery_time,min_delivery_time,max_delivery_time,total_deliveries,Buffet,Drinks,Meal,Snack
2022,3,Others,21.989461358313818,10,52,854,219,202,201,232
2022,4,Semi-Urban,49.32,44,54,25,5,6,8,6
2022,3,Urban,23.072326165875452,10,54,7162,1793,1818,1721,1830
2022,3,Semi-Urban,49.79661016949152,44,54,118,33,23,23,39
2022,4,Metropolitian,27.09486000835771,10,54,4786,1178,1198,1196,1214
2022,2,Semi-Urban,49.857142857142854,47,54,21,4,5,3,9
2022,4,Others,21.988165680473372,10,48,169,43,41,28,57
2022,3,Metropolitian,27.27553971913645,10,54,23855,5885,5934,6091,5945
2022,2,Others,22.45762711864407,10,54,177,41,43,46,47
2022,2,Metropolitian,27.682318415260454,10,54,5452,1364,1303,1377,1408


Databricks visualization. Run in Databricks to view.

**3. Factors affecting orders placed count**

In [0]:
# 3. Factors affecting orders placed count

# ---------------i) To get count of orders for Festive and non-festive seasons
# explicilty generating season_type column to track festival or non-festival season
df_with_season = df.withColumn(
    "season_type", 
    when(col("festival").like("%Yes%"), "festival").otherwise("non_festival")
)

# Pivot the data to get count of order  per year, month and season_type
festival_counts = df_with_season.groupBy('year', 'month', 'season_type').agg(
    count('id').alias('order_count')
)

# Pivot to get separate columns for each season_type
pivot_festival_counts = festival_counts.groupBy('year', 'month').pivot('season_type').agg(
    first('order_count')  # We use 'first' as there should be only one count per city/year/order type
)

# ---------------ii) To get order count for each season type
# Pivot the data to get count of order per year, month and weather_conditions
weather_conditions_counts = df.groupBy('year', 'month', 'weather_conditions').agg(
    count('id').alias('order_count')
)

# Pivot to get separate columns for each weather_conditions type
pivot_weather_conditions_counts = weather_conditions_counts.groupBy('year', 'month').pivot('weather_conditions').agg(
    first('order_count')  # We use 'first' as there should be only one count per city/year/order type
)

# ---------------iii) Orders count in different traffic density type
# Pivot the data to count of order per year, month and road_traffic_density
traffic_count = df.groupBy('year', 'month', 'road_traffic_density').agg(
    count('id').alias('order_count')
)

# Pivot to get separate columns for each road_traffic_density type
pivot_traffic_count = traffic_count.groupBy('year', 'month').pivot('road_traffic_density').agg(
    first('order_count')  # We use 'first' as there should be only one count per city/year/order type
)

# --------------Perform the join operations to combine all dataframes
final_df = pivot_festival_counts \
    .join(pivot_weather_conditions_counts, on=['year', 'month'], how='left') \
    .join(pivot_traffic_count, on=['year', 'month'], how='left')

final_df = final_df.drop("NaN") 

final_df.show()
display(final_df)

# Load the results to delta table
final_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("analytics.order_variations_by_condition")

+----+-----+-----+----+-----+-------+----+------+----+----------+------+-----+-----+--------+------------+
|year|month|High |Jam | Low |Medium |NaN |Cloudy| Fog|Sandstorms|Stormy|Sunny|Windy|festival|non_festival|
+----+-----+-----+----+-----+-------+----+------+----+----------+------+-----+-----+--------+------------+
|2022|    2|  686|2299| 2442|   1747|  68|  1211|1212|      1217|  1203| 1145| 1171|     148|        7094|
|2022|    3| 3083|9929|10883|   7644| 450|  5311|5331|      5218|  5343| 5155| 5181|     633|       31356|
|2022|    4|  656|1915| 2152|   1556|  83|  1014|1111|      1060|  1040|  984| 1070|     115|        6247|
+----+-----+-----+----+-----+-------+----+------+----+----------+------+-----+-----+--------+------------+



In [0]:
display(final_df)

year,month,High,Jam,Low,Medium,NaN,Cloudy,Fog,Sandstorms,Stormy,Sunny,Windy,festival,non_festival
2022,2,686,2299,2442,1747,68,1211,1212,1217,1203,1145,1171,148,7094
2022,3,3083,9929,10883,7644,450,5311,5331,5218,5343,5155,5181,633,31356
2022,4,656,1915,2152,1556,83,1014,1111,1060,1040,984,1070,115,6247


Databricks visualization. Run in Databricks to view.